# Character-level recurrent sequence-to-sequence model

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2017/09/29<br>
**Last modified:** 2020/04/26<br>
**Description:** Character-level recurrent sequence-to-sequence model.

## Introduction

This example demonstrates how to implement a basic character-level
recurrent sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

**Summary of the algorithm**

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.


## Setup


In [94]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from numpy.random import seed
seed(1)

import pickle
from tqdm import tqdm

## Download the data


In [95]:
# TUTORIAL DATA
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip


['Archive:  fra-eng.zip',
 'replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL',
 '(EOF or read error, treating as "[N]one" ...)']

In [96]:
# Indicate dataframes to import.
list_dfs = ['pickled_conala_mined_df', 'pickled_conala_train_df', 'pickled_conala_test_df',
           'conala_train_bag_df', 'conala_mined_bag_df', 'combined_bag_df']

In [97]:
%time
# Load all data in list_dfs
data = {}
for df in list_dfs:
    dbfile = open(df, 'rb')      
    contents = pickle.load(dbfile)
    data[df] = contents
    dbfile.close()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 18.1 µs


In [98]:
data.keys()

dict_keys(['pickled_conala_mined_df', 'pickled_conala_train_df', 'pickled_conala_test_df', 'conala_train_bag_df', 'conala_mined_bag_df', 'combined_bag_df'])

In [99]:
df = data["pickled_conala_train_df"]

In [100]:
df.head()

,intent,rewritten_intent,snippet,question_id
0,How to convert a list of multiple integers int...,Concatenate elements of a list 'x' of multiple...,"sum(d * 10 ** i for i, d in enumerate(x[::-1]))",41067960
1,How to convert a list of multiple integers int...,convert a list of integers into a single integer,"r = int(''.join(map(str, x)))",41067960
2,how to convert a datetime string back to datet...,convert a DateTime string back to a DateTime o...,datetime.strptime('2010-11-13 10:33:54.227806'...,4170655
3,Averaging the values in a dictionary based on ...,get the average of a list values for each key ...,"[(i, sum(j) / len(j)) for i, j in list(d.items...",29565452
4,zip lists in python,"zip two lists `[1, 2]` and `[3, 4]` into a lis...","zip([1, 2], [3, 4])",13704860


In [101]:
# There's some Nans in rewritten intent. Drop them.
drops = df[df["rewritten_intent"].isna()].index
df = df.drop(drops)

In [102]:
# prep takes lists of text.
input_list=list(df["snippet"])
target_list=list(df["rewritten_intent"])

In [103]:
target_list

["Concatenate elements of a list 'x' of multiple integers to a single integer",
 'convert a list of integers into a single integer',
 "convert a DateTime string back to a DateTime object of format '%Y-%m-%d %H:%M:%S.%f'",
 'get the average of a list values for each key in dictionary `d`)',
 'zip two lists `[1, 2]` and `[3, 4]` into a list of two tuples containing elements at the same index in each list',
 "prepend string 'hello' to all items in list 'a'",
 'regex for repeating words in a string `s`',
 'normalize a pandas dataframe `df` by row',
 'swap values in a tuple/list inside a list `mylist`',
 'Swap values in a tuple/list in list `mylist`',
 'find all occurrences of the pattern \'\\\\[[^\\\\]]*\\\\]|\\\\([^\\\\)]*\\\\)|"[^"]*"|\\\\S+\' within `strs`',
 'generate the combinations of 3 from a set `{1, 2, 3, 4}`',
 'add multiple columns `hour`, `weekday`, `weeknum` to pandas data frame `df` from lambda function `lambdafunc`',
 "BeautifulSoup search string 'Elsie' inside tag 'a'",
 '

## Configuration


In [104]:
batch_size = 128  # Batch size for training.
epochs = 20  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

## Prepare the data

This version will modify to tokenize by word instead of character on the INTENT side. The code side will STILL be character to character.

We'll do this by making modifications to the previously created character-to-character s2s code created previously.

Changes: 
- So we'll have `input_characters` and `target_vocab`
- target vocab needs to be by word instead of char.

In [105]:
import re
# Helper function will be needed in a moment.
def split_to_list_of_words(li):
    '''
    Takes a list of sentences.
    Returns a list of lists, each inner list is words in a sentence.
    '''
    
    new_list = list()
    for i in li:
        # Try to lowercase the sentence
        try:
            i = i.lower()
        except:
            pass
        
        # Add spaces on either side of special chars
        try:
            i = re.sub('([^a-zA-Z0-9 \d])', r' \1 ', i) # Add spaces between special chars
        except:
            pass # If this fails, move on. 
        
        # Split the sentence by spaces.
        i = list(i.split(' '))
        
        # Add this processed list of words to the new_list var, to be returned.
        new_list.append(i)
    return new_list

In [106]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_tokens = set()

# create a list of words from target_list with helper func.
target_list = split_to_list_of_words(target_list)

# Create lists of text by record to look through for vocab vars.
for input_text, target_text in zip(input_list, target_list):
    
    # Start target sentences with 't' and end with '\n'
    target_text.insert(0, "\t")
    target_text.append("\n")
    # previously implemented as following, since str not list.
        # target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    # Input vocab from input text list.
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
            
    # Target vocab from target text list
    for token in target_text:
        if token not in target_tokens:
            target_tokens.add(token)

In [107]:
# Peek at output
print("sample of input:\n\n", input_texts[:3], end="\n\n")
print("sample of target:\n\n", target_texts[:3], end="\n\n")
print("sample of input chars:\n\n", list(input_characters)[:10])

sample of input:

 ['sum(d * 10 ** i for i, d in enumerate(x[::-1]))', "r = int(''.join(map(str, x)))", "datetime.strptime('2010-11-13 10:33:54.227806', '%Y-%m-%d %H:%M:%S.%f')"]

sample of target:

 [['\t', 'concatenate', 'elements', 'of', 'a', 'list', '', "'", 'x', "'", '', 'of', 'multiple', 'integers', 'to', 'a', 'single', 'integer', '\n'], ['\t', 'convert', 'a', 'list', 'of', 'integers', 'into', 'a', 'single', 'integer', '\n'], ['\t', 'convert', 'a', 'datetime', 'string', 'back', 'to', 'a', 'datetime', 'object', 'of', 'format', '', "'", '', '%', 'y', '-', '', '%', 'm', '-', '', '%', 'd', '', '%', 'h', ':', '', '%', 'm', ':', '', '%', 's', '.', '', '%', 'f', "'", '', '\n']]

sample of input chars:

 ['+', 'F', 'Z', 'D', '|', '.', '}', 'L', '4', 'X']


In [111]:
# Get simple size metrics, and sort vocabs alphabetically.
input_characters = sorted(list(input_characters))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_tokens)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

# Output findings of above.
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

# Creates a vocabulary dictionary, with token and enum int.
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(token, i) for i, token in enumerate(target_tokens)])
target_token_index.update( {' ' : len(target_token_index)} )

# Sets up the array shapes for input/output data with zeroed arrays.
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

# Loop through the corpus, which is list of sentences for input,
# and a list of sentences (sentences as lists of words) for target.
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # loop through chars in input text, 1 hot encode character by character.
    # The resulting array will be the input data.
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    # end the sentence with " " character
    # encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    # end the sentence with " " character
    # decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    # end the sentence with " " character
    # decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 2300
Number of unique input tokens: 98
Number of unique output tokens: 2433
Max sequence length for inputs: 232
Max sequence length for outputs: 98


## Build the model


In [112]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


## Train the model


In [114]:
import os
%time
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

if not os.path.exists('models/s2s'):
    model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2,
    )
    # Save model
    model.save('models/s2s')

else:
    print("No need to train, model can be loaded." \
          " Move on to next step...")
    

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs
Epoch 1/20
15/15 [==============================] - 83s 6s/step - loss: 1.1496 - accuracy: 0.7785 - val_loss: 0.9856 - val_accuracy: 0.8339
Epoch 2/20
15/15 [==============================] - 88s 6s/step - loss: 0.9422 - accuracy: 0.8345 - val_loss: 0.9771 - val_accuracy: 0.8323
Epoch 3/20
15/15 [==============================] - 94s 6s/step - loss: 0.9286 - accuracy: 0.8350 - val_loss: 0.9702 - val_accuracy: 0.8339
Epoch 4/20
15/15 [==============================] - 108s 7s/step - loss: 0.9210 - accuracy: 0.8353 - val_loss: 0.9742 - val_accuracy: 0.8281
Epoch 5/20
15/15 [==============================] - 101s 7s/step - loss: 0.9138 - accuracy: 0.8354 - val_loss: 0.9718 - val_accuracy: 0.8339
Epoch 6/20
15/15 [==============================] - 122s 8s/step - loss: 0.9083 - accuracy: 0.8366 - val_loss: 0.9710 - val_accuracy: 0.8359
Epoch 7/20
15/15 [==============================] - 81s 5s/step - loss: 0.9029 - accuracy: 0.

## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states


In [119]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("models/s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence = decoded_sentence + ' ' + sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


You can now generate decoded sentences as such:


In [120]:
for seq_index in range(5):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


-
Input sentence: sum(d * 10 ** i for i, d in enumerate(x[::-1]))
Decoded sentence:  convert a list  `  `  `  `  `  `  `  `  

-
Input sentence: r = int(''.join(map(str, x)))
Decoded sentence:  convert a list  `  `  `  `  `  `  `  `  

-
Input sentence: datetime.strptime('2010-11-13 10:33:54.227806', '%Y-%m-%d %H:%M:%S.%f')
Decoded sentence:  convert a list  `  `  `  `  `  `  `  `  

-
Input sentence: [(i, sum(j) / len(j)) for i, j in list(d.items())]
Decoded sentence:  convert a list  `  `  `  `  `  `  `  `  

-
Input sentence: zip([1, 2], [3, 4])
Decoded sentence:  convert a list  `  `  `  `  `  `  `  `  



# Testing with Prob Char Selection

Previously, used argmax

In [121]:
# Testing output cell
input_seq = encoder_input_data[0 : 0 + 1]
states_value = encoder_model.predict(input_seq)

In [122]:
input_seq

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)

In [126]:
states_value[0].shape
target_seq = np.zeros((1, 1, num_decoder_tokens))
target_seq[0, 0, target_token_index["\t"]] = 1.0
stop_condition = False
decoded_sentence = ""

# output_tokens is the prob list
predict_proba, h, c = decoder_model.predict([target_seq] + states_value)

sampled_char = np.random.choice(target_tokens, 1, p = predict_proba.flatten())[0]
new = predict_proba.flatten()
new.sort()
new[-1::-1]
#print(predict_proba[-1],predict_proba[-2])

array([1.0067537e-01, 8.9342199e-02, 7.9250462e-02, ..., 6.8802393e-07,
       6.8789728e-07, 6.8132141e-07], dtype=float32)

In [129]:
# Use decode sequence with random choice on predicted probs. 
def decode_sequence_prob(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        predict_proba, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_char = np.random.choice(target_tokens, 1, p = predict_proba.flatten())[0]
        sampled_token_index = target_tokens.index(sampled_char)
        
        decoded_sentence = decoded_sentence + ' ' + sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [135]:
# Testing probability selection
for seq_index in range(5):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence_prob(input_seq) # change decode func
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)
    print("Actual intent:", ' '.join(target_texts[seq_index]))


-
Input sentence: sum(d * 10 ** i for i, d in enumerate(x[::-1]))
Decoded sentence:  import get  in if list  `  ` data 

Actual intent: 	 concatenate elements of a list  ' x '  of multiple integers to a single integer 

-
Input sentence: r = int(''.join(map(str, x)))
Decoded sentence:  dictionary the string  in url model listd  container items ends of  in new 

Actual intent: 	 convert a list of integers into a single integer 

-
Input sentence: datetime.strptime('2010-11-13 10:33:54.227806', '%Y-%m-%d %H:%M:%S.%f')
Decoded sentence:  count list  in df list with `  from color where from xcc and contains 

Actual intent: 	 convert a datetime string back to a datetime object of format  '  % y -  % m -  % d  % h :  % m :  % s .  % f '  

-
Input sentence: [(i, sum(j) / len(j)) for i, j in list(d.items())]
Decoded sentence:  extract - 123 of  ` on each key  ` b with  

Actual intent: 	 get the average of a list values for each key in dictionary  ` d `  )  

-
Input sentence: zip([1, 2], [3